# Molecular Integrals

In this tutorial we are going to see how to implement the _McMurchie-Davidson scheme_ to compute molecular integrals over _Gaussian functions_ using the [Julia programming language](https://julialang.org/).  
We are going to closely follow Chapter 9 of the book _Molecular Electronic Structure Theory_ by _T. Helgaker et al._, although the inspiritation for this work comes from [this post](http://joshuagoings.com/2017/04/28/integrals/) and thus credits go to the author of the latter as well.

In traditional quantum chemical calculations we are essentially interested in the evaluation of two classes of molecular integrals.  
The first class consists of integrals over _one-electron operators_ $\hat{O}(\mathbf{r})$, such as in the case of overlap, kinetic energy and electron-nuclear Coulomb attraction integrals.  
These integrals have the general form

$$
O_{\mu\nu} = \int \chi_{\mu}(\mathbf{r}) \hat{O}(\mathbf{r}) \chi_{\nu}(\mathbf{r}) d\mathbf{r}
$$

The second class consists of integrals over _two-electron operators_ $\hat{O}(\mathbf{r}_1,\mathbf{r}_2)$, in particular the electron-electron Coulomb repulsion integral, with the general form given by

$$
O_{\mu\nu\lambda\sigma} = \int\int \chi_{\mu}(\mathbf{r}_1)\chi_{\nu}(\mathbf{r}_1)\hat{O}(\mathbf{r}_1,\mathbf{r}_2)\chi_{\lambda}(\mathbf{r}_2)\chi_{\sigma}(\mathbf{r}_2) d\mathbf{r}_1 d\mathbf{r}_2
$$

In both cases, the integrals are computed over a set of known functions $\{\chi_{\mu}\}$, which are usually chosen to be linear combinations of Gaussian functions.
This choice is arbitrary (in principle any type of function can be used), but it is motivated on a firm theoretical ground and has proven to be a very successful strategy.

There are different types of Guassian functions, each featuring (slightly) different properties.
In order to efficientely evaluate the integrals and handle them in electronic structure calculations, we shall use the form of Gaussian which is most convenient at every step of the procedure.

In the following we shall use the generic term _Gaussian-type orbital (GTO)_ when talking about _a single_ Gaussian function, irrespective of its particular type: this will always be clear from the context or explicitly specified otherwise.

## Cartesian Gaussian Functions

Because of their simplicity and properties, _Cartesian Gaussian functions (CGFs)_, also called _Cartesian GTOs_, are used in nearly all calculations on polyatomic systems. They are given by

$$
G_{ijk}(\mathbf{r},\alpha,\mathbf{A}) = x_A^i y_A^j z_A^k e^{-\alpha r_A^2}
$$

The position of the electron is defined relative to the nucleus centered on $\mathbf{A}$

$$
\mathbf{r}_A = \mathbf{r} - \mathbf{A}
$$

and $x_A, y_A$ and $z_A$ are the relative Cartesian components (note that **boldface** symbols represent vector quantities). The _Gaussian exponent_ $\alpha$ is a real number greater than zero, while the _Cartesian quantum numbers_ $i,j$ and $k$ are integers greater than or equal to zero. The total angular momentum is given by the following expression

$$
l = i + j + k
$$

and the set of all possible combinations $(i,j,k)$ with total angular momentum $l$ is said constitute a _shell_.
The number of GTOs in a shell of total angular momentum $l$ is given by

$$
N_l^c = \frac{(l+1)(l+2)}{2}
$$

Commonly, in electronic structure calculation, one uses _spherical-harmonic Gaussian functions (SGFs)_ since their number in a given shell is always smaller than or equal to the number of CGFs ($1,3,5,7,9,\ldots$ vs $1,3,6,10,15,\ldots$), albeit spanning the same space.
One therefore would preferably work with SGFs rather than CGFs so as to keep their number as small as possible. Nevertheless, contrary to SGFs, CGFs are separable in the three Cartesian components, a fact that substantailly facilitates the calculation of integrals.
Therefore, the general approach is to evaluate molecular integrals over CGFs first, and then transform the integrals to the SGFs basis in order to decrease their total number.
We are going to see this transformation at the very end of this tutorial.

Let us start the implementation by defining an _abstract type_ representing the concept of a _basis function_

In [1]:
"""Abstract type representing a basis function."""
abstract type BasisFunction end

BasisFunction

and now a concrete type representing a Cartesian GTO

In [2]:
"""Concrete type defining a Cartesian Gaussian function."""
struct CGF <: BasisFunction
    A::NTuple{3,Float64}    # center of the CGF
    α::Float64              # exponent
    ijk::NTuple{3,Int}      # Cartesian quantum numbers
end


"""`CGF` constructor using a vector as the center."""
function CGF(A::AbstractVector{T}, α::Real, ijk::NTuple{3,Int}) where {T<:Real}
    return CGF((A[1],A[2],A[3]),α,ijk)
end

CGF

Furthermore, we would like to have some getter functions to easily access the attributes of a `CGF`

In [3]:
center(gto::CGF)   = [gto.A[1],gto.A[2],gto.A[3]]
exponent(gto::CGF) = gto.α
shell(gto::CGF)    = gto.ijk

shell (generic function with 1 method)

As mentioned above, CGFs may be factorized in the three Cartesian directions, i.e.

$$
G_a(\mathbf{r},\alpha,\mathbf{A}) = G_{ijk}(\mathbf{r},\alpha,\mathbf{A}) = G_i(x,\alpha,A_x) G_j(y,\alpha,A_y) G_k(z,\alpha,A_z)
$$

where a one-dimensional component has the form

$$
G_i(x,\alpha,A_x) = x_A^i e^{-\alpha x_A^2}
$$

The self-overlap integral over such one-dimensional component is given by

$$
\langle G_i|G_i \rangle = \frac{(2i-1)!!}{(4\alpha)^i}\sqrt{\frac{\pi}{2\alpha}}
$$

with the double factorial function defined as follows

$$
n!! =
\begin{cases}
1 & n=0 \\
n(n-2)(n-4)\cdots 2 & \text{even } n > 0 \\
n(n-2)(n-4)\cdots 1 & \text{odd } n > 0 \\
\frac{1}{(n+2)(n+4)\cdots 1} & \text{odd } n < 0
\end{cases}
$$

Clearly, the self-overlap integral over the three dimensions is just the multiplication of the self-overlap integral of the individual components

$$
\begin{aligned}
\langle G_a|G_a \rangle &= \langle G_{ijk}|G_{ijk} \rangle \\
&= \langle G_i|G_i \rangle \langle G_j|G_j \rangle \langle G_k|G_k \rangle \\
&= \frac{(2i-1)!!(2j-1)!!(2k-1)!!}{(4\alpha)^{i+j+k}}\left(\frac{\pi}{2\alpha}\right)^{\tfrac{3}{2}}
\end{aligned}
$$

In general, we shall always work with _unnormalized_ basis functions, nevertheless, it will be useful later on to have a function that returns the normalization factor of a GTO. This is trivially given by the inverse square root of the self-overlap. Let's see how to do that.  
First, we define a function computing the double factorial introduced above

In [4]:
"""Returns the double factorial n!! of an integer number."""
function dfactorial(n::Int)
    if n == 0
        return 1.0
    elseif iseven(n) && n > 0
        return reduce(*,n:-2:2)[end]
    elseif isodd(n) && n > 0
        return reduce(*,n:-2:1)[end]
    elseif isodd(n) && n < 0
        return 1/reduce(*,n+2:2:1)[end]
    else
        error("n!! undefined for negative even values")
    end
end

dfactorial

With the help of `dfactorial`, we can now implement a function to compute the self-overlap as well as a function named `normalization` which computes the inverse square root of the overlap and returns the normalization factor of a CGF

In [5]:
"""Returns the self-overlap integral <Ga|Ga> of a `CGF` Ga."""
function overlap(Ga::CGF)
    α = exponent(Ga)
    (i,j,k) = shell(Ga)
    num = dfactorial(2*i-1)*dfactorial(2*j-1)*dfactorial(2*k-1)
    den = (4.0*α)^(i+j+k)
    return ((π/(2.0*α))^1.5 * num)/den
end


"""Returns the normalization factor of a `CGF` Ga."""
function normalization(Ga::CGF)
    α = exponent(Ga)
    (i,j,k) = shell(Ga)
    num = (2.0*α/π)^0.75 * (4.0*α)^((i+j+k)/2.0)
    den = sqrt(dfactorial(2*i-1)*dfactorial(2*j-1)*dfactorial(2*k-1))
    return num/den
end

normalization

### Gaussian product rule

One of the main advantages of working with Gussian functions is the _Gaussian product rule_.
The product of two spherical Gaussians centered on $\mathbf{A}$ and $\mathbf{B}$ may be written in terms of a single Gaussian located between the two centers, thus reducing the working complexity. This can be seen from the following expression

$$
e^{-\alpha x_A^2} e^{-\beta x_B^2} = e^{-\mu X_{AB}^2} e^{-px_P^2} = K_{\alpha\beta}^x e^{-px_P^2}
$$

The _total exponent_ $p$ and the _reduced exponent_ $\mu$ are given by

$$
\begin{aligned}
p &= \alpha + \beta \\
\mu &= \frac{\alpha\beta}{\alpha + \beta}
\end{aligned}
$$

whereas the _centre-of-charge coordinate_ $P_x$ and the _Guassian separation_ $X_{AB}$ read

$$
\begin{aligned}
P_x &= \frac{\alpha A_x + \beta B_x}{\alpha + \beta} = \frac{\alpha A_x + \beta B_x}{p} \\
X_{AB} &= A_x - B_x
\end{aligned}
$$

The _pre-exponential factor_ $K_{\alpha\beta}^x$ does not depend on the elctronic coordinate $x$ and it is given by

$$
K_{\alpha\beta}^x = e^{-\mu X_{AB}^2}
$$

### Gaussian overlap distributions

Combining the Gaussian product rule with the fact that Cartesian Gaussians factorize along the three directions, allows us to handle _Gaussian overlap distributions_, i.e. products of Gaussians, in a easier way.  
We introduce now a short-hand notation for Cartesiano GTOs that we shall use throughout this tutorial

$$
\begin{aligned}
G_{a}(\mathbf{r}) &= G_{ikm}(\mathbf{r},\alpha,\mathbf{A}) \\
G_{b}(\mathbf{r}) &= G_{jln}(\mathbf{r},\beta ,\mathbf{B})
\end{aligned}
$$

and we define the _Gaussian overlap distribution_ of $G_{a}(\mathbf{r})$ and $G_{b}(\mathbf{r})$ as

$$
\Omega_{ab}(\mathbf{r}) = G_{a}(\mathbf{r})G_{b}(\mathbf{r})
$$

The overlap distribution may be factorized along Cartesian directions as

$$
\Omega_{ab}(\mathbf{r}) = \Omega_{ij}^x(x,\alpha,\beta,A_x,B_x) \Omega_{kl}^y(y,\alpha,\beta,A_y,B_y) \Omega_{mn}^z(z,\alpha,\beta,A_z,B_z)
$$

where individual components are given by

$$
\Omega_{ij}^x(x,\alpha,\beta,A_x,B_x) = G_i(x,\alpha,A_x) G_j(x,\beta,B_x)
$$

As the last step, we may now apply the Gaussian product rule and write the overlap distribution as a single-center Gaussian function as follows

$$
\Omega_{ij}^x = K_{\alpha\beta}^x x_A^i x_B^j e^{-px_P^2}
$$

which ends for the moment our discussion on Cartesian GTOs.

We should point out at this stage that we have all necessary ingredients to compute simple one-electron integrals using the _Obara-Saika scheme_, however, as we are true adventurers, we shall continue to dive deeper into the topic and aim for the McMurchie-Davidson scheme, which requires another type of Gaussian functions.

## Hermite Gaussian functions

_Hermite Gaussian functions_, or simply _Hermite Gaussians_, are another type of Gaussians which are very similar to Cartesian GTOs.  
A nice property of Cartesian GTOs is that their derivatives are connected through _recurrence relations_ involving functions of different angular momenta. This will turn out to be particularly useful for two reasons. First, in the calculation of molecular properties we need to compute derivatives of the Hamiltonian and thus also of the integrals. Second, the relation between Gaussians of different angular momenta allows for the calculation of integrals over GTOs with higher angular components by reusing the integrals already computed for lower ones, thus avoiding the explicit calculations of complicated integrals.  
Hermite Gaussians offer recurrence relations similar to those available for Cartesian GTOs, which are exploited in the McMurchie-Davidson scheme to expand Gaussian overlap distributions in one-centre Hermite Gaussians.

Let us introduce the Hermite Gaussian with exponent $p$ and center $\mathbf{P}$

$$
\Lambda_{tuv}(\mathbf{r},p,\mathbf{P}) = \left(\frac{\partial}{\partial P_x}\right)^t \left(\frac{\partial}{\partial P_y}\right)^u \left(\frac{\partial}{\partial P_z}\right)^v e^{-p r_P^2}
$$

where $\mathbf{r}_P = \mathbf{r} - \mathbf{P}$.  
Like for Cartesian GTOs, Hermite Gaussians are separable

$$
\Lambda_{tuv}(\mathbf{r},p,\mathbf{P}) = \Lambda_{t}(x,p,P_x) \Lambda_{u}(y,p,P_y) \Lambda_{v}(z,p,P_z)
$$

where the subscripts $t, u, v$ denote the degree of the polynomial factor in front of the Gaussian, as can be seen from the first expression for $\Lambda_{tuv}$ above.  
An important result that we are going to use later on in this tutorial is the value of the one-dimensional integral over the entire (one-dimensional) space, given by

$$
\int \Lambda_t dx = \delta_{t0}\sqrt{\frac{\pi}{p}}
$$

where we note that the only non-zero case is for $t = 0$ because of the presence of the Dirac delta function $\delta_{t0}$.  
As far as this tutorial goes, the above expression is the only relation we _explicitly_ require in order to continue with the McMurchie-Davidson scheme as presented here. However, note that many results obtained in the next sections are based on recurrence relations of Hermite Gaussians which are _not explicitly_ shown here.

## The McMurchie-Davidson scheme

We shall now see how Hermite Gaussians are used within the McMurchie-Davidson scheme. Because integrals over Hermite Gaussians are so simple, it is convenient to express Cartesian Gaussian distributions in terms of Hermite Gaussian funcntions, such that their integration is faster.

### Expanding Cartesian Gaussian overlap distributions in Hermite Gaussians 

Consider the 1D Cartesian overlap distribution $\Omega_{ij}^x$ centered at $P$, which is a polynomial of degree $i+j$ in $x_P$.
We can expand $\Omega_{ij}^x$ in a linear combination of Hermite Gaussian functions _exactly_, according to

$$
\Omega_{ij}^x = \sum_{t=0}^{i+j} E_t^{ij} \Lambda_t
$$

where we point out that the coefficients $E_t^{ij}$ have no dependence on $x_P$.  
Through the application of a number of Gaussian properties and algebraic manipulations, it is possible to derive recurrence relations that determine the expansion coefficients $E_t^{ij}$ exactly.
One such two-term recursive relation is given by

$$
\begin{aligned}
    E_t^{ij} &= 0 \, , \quad t < 0 \quad \text{or} \quad t > i+j \\
    E_0^{00} &= K_{\alpha\beta}^x \\
    E_0^{i+1,j} &= X_{PA} E_0^{ij} + E_1^{ij} \\
    E_0^{i,j+1} &= X_{PB} E_0^{ij} + E_1^{ij} \\
    E_t^{i,j} &= \frac{1}{2pt}(iE_{t-1}^{i-1,j} + jE_{t-1}^{i,j-1})
\end{aligned}
$$

with $X_{PA} = P_x - A_x$ and $X_{PB} = P_x - B_x$.  
We recall the expression for $K_{\alpha\beta}^x$, which starts the recursion, reading

$$
K_{\alpha\beta}^x = e^{-\mu X_{AB}^2}
$$

where $\mu = \alpha\beta/(\alpha+\beta)$ and $X_{AB} = A_x - B_x$.  
The expansion coefficients thus depend on the two exponents, the two Gaussian centers as well as their distance from the overlap centre of mass.

Conveniently, it is possible to implement recursive formulas in Julia without any problem!  
Let's define a function that computes the Hermite expansion coefficients of a 1D Cartesian overlap distribution

In [6]:
"""
Returns the Hermite expansion coefficients for a 1D Cartesian overlap distribution
using a two-term recursion relation.
"""
function Etij(t::Int ,i::Int, j::Int, Kαβx::Real, XPA::Real, XPB::Real, α::Real, β::Real)
    
    # compute overlap exponent
    p = α + β
    
    # enter recursion
    if t < 0 || t > i+j
        return 0.0
    elseif t == 0
        if i == j == 0
            return Kαβx
        elseif j == 0
            return XPA * Etij(0, i-1, j, Kαβx, XPA, XPB, α, β) +
                         Etij(1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return XPB * Etij(0, i, j-1, Kαβx, XPA, XPB, α, β) +
                         Etij(1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    else
        return (1/(2*p*t)) * (i * Etij(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                              j * Etij(t-1, i, j-1, Kαβx, XPA, XPB, α, β) )
    end
end

Etij

Notice that there exist other recursion relations to compute the expansion coefficients. An alternative relation is presented at the end of this tutorial in the Appendix section.
Here, we use the two-term relation implemented above which appears to be the most efficient (at least in terms of operations count).

At this point, we are finally ready to evaluate the first integrals!

### Overlap integrals

Let us consider the following 1D overlap integral

$$
S_{ij} = \langle G_i|G_j \rangle = \int G_i G_j dx = \int \Omega_{ij}^x dx
$$

We can substitute $\Omega_{ij}^x$ with the Hermite expansion illustrated above, obtaining

$$
\int \Omega_{ij}^x dx = \sum_{t=0}^{i+j} E_t^{ij} \int \Lambda_t dx = \sum_{t=0}^{i+j} E_t^{ij} \delta_{t0} \sqrt{\frac{\pi}{p}} = E_0^{ij} \sqrt{\frac{\pi}{p}}
$$

Generalizing this expression to the 3D case results in the McMurchie-Davidson formula for the overlap integral between two GTOs, given by

$$
\begin{aligned}
S_{ab} &= \langle G_a | G_b \rangle \\
       &= \langle G_{ikm} | G_{jln} \rangle \\
       &= \langle G_i|G_j \rangle \langle G_k|G_l \rangle \langle G_m|G_n \rangle \\
       &= S_{ij} S_{kl} S_{mn} \\
       &= E_0^{ij} E_0^{kl} E_0^{mn} \left(\sqrt{\frac{\pi}{p}}\right)^3 \\
       &= E_0^{ij} E_0^{kl} E_0^{mn} \left(\frac{\pi}{p}\right)^{\tfrac{3}{2}}
\end{aligned}
$$

We can now implement a function computing the overlap integral between two Cartesian Gaussian fuctions represented by the `CGF` type defined at the beginning of the tutorial

In [7]:
"""Returns the overlap integral <Ga|Gb> of two Cartesian GTOs."""
function overlap(Ga::CGF, Gb::CGF)
    
    # precomputing all required quantities
    α = exponent(Ga); β = exponent(Gb)
    A = center(Ga);   B = center(Gb)
    μ = (α * β)/(α + β)
    P = (A*α + B*β)./(α + β)
    RAB = A - B; RPA = P - A; RPB = P - B
    Kαβ = exp.(-μ*RAB.^2)
    (ia,ja,ka) = shell(Ga)
    (ib,jb,kb) = shell(Gb)
    
    # calculate overlaps in the 3 Cartesian directions
    Sx = Etij(0, ia, ib, Kαβ[1], RPA[1], RPB[1], α, β)
    Sy = Etij(0, ja, jb, Kαβ[2], RPA[2], RPB[2], α, β)
    Sz = Etij(0, ka, kb, Kαβ[3], RPA[3], RPB[3], α, β)
    
    return Sx * Sy * Sz * (π / (α + β) )^1.5
end

overlap

This is a good moment to test the functions implemented so far

In [8]:
Ga = CGF( ones(3), 0.3, (0,0,0)) # s   orbital centered at [1,1,1]
Gb = CGF(zeros(3), 0.5, (0,1,0)) # py  orbital centered at origin
Gc = CGF(zeros(3), 0.2, (0,1,0)) # py  orbital centered at origin
Gd = CGF(zeros(3), 0.75,(0,1,1)) # dxz orbital centered at origin

println("<Ga|Ga> - <Ga|Ga> = ", overlap(Ga) - overlap(Ga,Ga)) # -> 0.0
println("<Gd|Gd> - <Gd|Gd> = ", overlap(Gd) - overlap(Gd,Gd)) # -> 0.0
println("<Ga|Gb> = ",overlap(Ga,Gb))  # -> 1.662763376131468
println("<Ga|Gd> = ",overlap(Ga,Gd))  # -> 0.22213421730795865
println("<Gb|Gc> = ",overlap(Gb,Gc))  # -> 6.79124992650095
println("<Gc|Gb> = ",overlap(Gc,Gb))  # -> = <Gb|Gc> = 6.79124992650095
println("<Gb|Gd> = ",overlap(Gb,Gd))  # -> 0.0

<Ga|Ga> - <Ga|Ga> = 0.0
<Gd|Gd> - <Gd|Gd> = 0.0
<Ga|Gb> = 1.662763376131468
<Ga|Gd> = 0.22213421730795865
<Gb|Gc> = 6.79124992650095
<Gc|Gb> = 6.79124992650095
<Gb|Gd> = 0.0


Note that GTOs centered on the same position in space but with different angular momenta (even in the same shell!)  are orthogonal to each other and therefore their overlap should be eaxctly zero!
On the other hand, if their centers are different, their overlap will be $\neq 0$.

### Kinetic energy integrals

We continue with the evaluation of kinetic energy integrals by defining the general integral over differential operators

$$
D_{ab}^{efg} = \langle G_a |
\frac{\partial^e}{\partial x^e} \frac{\partial^f}{\partial y^f} \frac{\partial^g}{\partial z^g}
| G_b \rangle
$$

Again, we can factorize these integrals, such that in the following we can focus on the one-dimensional case exemplified by

$$
D_{ij}^e = \langle G_i | \frac{\partial^e}{\partial x^e} | G_j \rangle
$$

Thanks to the differentiation properties of (Cartesian) Gaussians, it is possible to express integrals over differential operators in terms of simple overlaps. In particular, for the second derivative with respect to the electronic coordinate, $D_{ij}^2$, we have

$$
D_{ij}^2 = 4\alpha^2 S_{i+2,j} - 2\alpha(2i +1)S_{ij} + i(i-1) S_{i-2,j}
$$

Accordingly, the three-dimensional case is given by

$$
\langle G_a|\frac{\partial^2}{\partial x^2}|G_b \rangle =
\langle G_i|\frac{\partial^2}{\partial x^2}|G_j \rangle
\langle G_k|G_l \rangle \langle G_m|G_n \rangle = D_{ij}^2 S_{kl} S_{mn}
$$

However, the kinetic energy integral over two GTOs reads

$$
T_{ab} = -\frac{1}{2} \langle G_a|\nabla^2|G_b \rangle
$$

Therefore, by expanding the (Laplace) operator $\nabla^2$ ($= \Delta$) explicitly, we can obtain the following expression

$$
\begin{aligned}
T_{ab} &= -\frac{1}{2} \langle G_a|\frac{\partial^2}{\partial x^2} +
\frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}|G_b \rangle \\
&= -\frac{1}{2} (\langle G_a|\frac{\partial^2}{\partial x^2}|G_b \rangle +
\langle G_a|\frac{\partial^2}{\partial y^2}|G_b \rangle +
\langle G_a|\frac{\partial^2}{\partial z^2}|G_b \rangle) \\
&= -\frac{1}{2}(D_{ij}^2S_{kl}S_{mn} + S_{ij}D_{kl}^2S_{mn} + S_{ij}S_{kl}D_{mn}^2 )
\end{aligned}
$$

The kinetic energy integrals are computed relying exclusively on overlap integrals and therefore their implementation is quite straightforward.  
However, since we require overlaps of arbitrary angular momenta, it is more convenient to define a _second_ overlap integral function accepting different input parameters than the one previously defined. We can call it with the same name `overlap`, and a different signature (i.e. input parameters) is sufficient for Julia multiple dispatch system to understand which function to call

In [9]:
"""Returns the overlap integral <Ga|Gb> of two Cartesian Gaussian functions."""
function overlap(α::Real, ikm::Tuple{Int,Int,Int}, A::Vector{T},
                 β::Real, jln::Tuple{Int,Int,Int}, B::Vector{T}) where {T<:Real}
    
    # precomputing all required quantities
    μ = (α * β)/(α + β)
    P = (A*α + B*β)./(α + β)
    RAB = A - B; RPA = P - A; RPB = P - B
    Kαβ = exp.(-μ*RAB.^2)
    
    # calculate overlaps in the 3 Cartesian directions
    Sx = Etij(0, ikm[1], jln[1], Kαβ[1], RPA[1], RPB[1], α, β)
    Sy = Etij(0, ikm[2], jln[2], Kαβ[2], RPA[2], RPB[2], α, β)
    Sz = Etij(0, ikm[3], jln[3], Kαβ[3], RPA[3], RPB[3], α, β)
    
    return Sx * Sy * Sz * (π / (α + β) )^1.5
end


"""Returns the kinetic energy integral -0.5*<Ga|∇^2|Gb> of two CGFs."""
function kinetic(Ga::CGF, Gb::CGF)
    
    # extract info from CGFs
    α = exponent(Ga); β = exponent(Gb)
    A = center(Ga); B = center(Gb)
    (i,k,m) = shell(Ga); (j,l,n) = shell(Gb)
    
    # precompute common terms
    dαSab = 2*α*overlap(α, (i,k,m), A, β, (j,l,n), B)
    fα2 = 4*α^2
    
    # Dij^2 * Skl * Smn
    Dij = i*(i-1) * overlap(α, (i-2,k,m), A, β, (j,l,n), B) -
          (2*i+1) * dαSab +
              fα2 * overlap(α, (i+2,k,m), A, β, (j,l,n), B)
    
    # Sij * Dkl^2 * Smn
    Dkl = k*(k-1) * overlap(α, (i,k-2,m), A, β, (j,l,n), B) -
          (2*k+1) * dαSab +
              fα2 * overlap(α, (i,k+2,m), A, β, (j,l,n), B)
    
    # Sij * Skl * Dmn^2
    Dmn = m*(m-1) * overlap(α, (i,k,m-2), A, β, (j,l,n), B) -
          (2*m+1) * dαSab +
              fα2 * overlap(α, (i,k,m+2), A, β, (j,l,n), B)
    
    return -0.5 * (Dij + Dkl + Dmn)
end

kinetic

Note that, from a computational point of view, it would be more convenient to wrap the calculation of overlap and kinetic energy integrals together in a single function or precompute the Hermite expansion coefficients $E^{ij}_t$, since the former are used for the latter, avoiding unnecessary recalculation of common terms.

### Nuclear attraction integrals

To complete the one-electron integrals section we now discuss the nuclear Coulomb attraction integrals.  
They differ from the previous cases in the fact that these integrals cannot be factorized along the three Cartesian directions.  
We are interested in evaluating the following integral

$$
V_{ab} = \langle G_a|\frac{1}{r_C}|G_b \rangle = \int \frac{\Omega_{ab}(\mathbf{r})}{r_C}d\mathbf{r}
$$

where $r_C$ is the distance between the electron and the nucleus position $\mathbf{C}$, while $\Omega_{ab}(\mathbf{r})$ is the Cartesian Gaussian overlap distribution arising from the two GTOs involved in the integral.
As before, the idea is to expand $\Omega_{ab}$ in Hermite Gaussian functions as

$$
V_{ab} = \sum_{tuv} E_{tuv}^{ab} \int \frac{\Lambda_{tuv}(\mathbf{r})}{r_C}d\mathbf{r}
$$

where $E_{tuv}^{ab} = E_t^{ij}E_u^{kl}E_v^{mn}$.

The _Hermite Coulomb integral_ on the right-hand side can be evaluated as follows

$$
\int \frac{\Lambda_{tuv}(\mathbf{r})}{r_C}d\mathbf{r} = \frac{2\pi}{p} \left(\frac{\partial}{\partial P_x}\right)^t \left(\frac{\partial}{\partial P_y}\right)^u \left(\frac{\partial}{\partial P_z}\right)^v F_0(pR_{PC}^2)
$$

where $F_0(x)$ is the special case $n=0$ of the _Boys function_ $F_n(x)$, given by

$$
F_n(x) = \int_0^1 e^{-xt^2}t^{2n} dt
$$

We shall see later on how to efficiently evaluate the Boys function.  
If $t,u$ or $v$ are different from zero, the differential operators appearing in the Hermite Coulomb integral act on the Boys function. However, such derivatives can be conveniently expressed in terms of Boys functions with a higher value of $n$.  
In general, Hermite integrals for $t+u+v \le N$ may be calculated from Boys functions $F_n$ of order $n \le N$. To this end, let us introduce the _auxiliary Hermite integrals_

$$
R^n_{tuv}(p,\mathbf{R}_{PC}) = \left(\frac{\partial}{\partial P_x}\right)^t \left(\frac{\partial}{\partial P_y}\right)^u \left(\frac{\partial}{\partial P_z}\right)^v R^n_{000}(p,R_{PC})
$$

where

$$
R^n_{000}(p,R_{PC}) = (-2p)^n F_n(pR_{PC}^2)
$$

By exploiting properties of the Boys function, it is possible to derive the following recursive relations for the evaluation of the auxiliary integrals $R^n_{tuv}(p,\mathbf{R}_{PC})$

$$
\begin{aligned}
R^n_{t+1,u,v}(p,\mathbf{R}_{PC}) = t R^{n+1}_{t-1,u,v}(p,\mathbf{R}_{PC}) + X_{PC} R^{n+1}_{tuv}(p,\mathbf{R}_{PC}) \\
R^n_{t,u+1,v}(p,\mathbf{R}_{PC}) = u R^{n+1}_{t,u-1,v}(p,\mathbf{R}_{PC}) + Y_{PC} R^{n+1}_{tuv}(p,\mathbf{R}_{PC}) \\
R^n_{t,u,v+1}(p,\mathbf{R}_{PC}) = v R^{n+1}_{t,u,v-1}(p,\mathbf{R}_{PC}) + Z_{PC} R^{n+1}_{tuv}(p,\mathbf{R}_{PC})
\end{aligned}
$$

We finally have an expression for $V_{ab}$ given by

$$
V_{ab} = \frac{2\pi}{p} \sum_{tuv} E_{tuv}^{ab} R^0_{tuv}(p,\mathbf{R}_{PC})
$$

Note that the nuclear attraction part of the Hamiltonian contains contributions from all the nuclei and their charge. Therefore, in practice we will have to calculate

$$
h_{ab} = -\sum_K Z_K V_{ab}(\mathbf{C}_K) = -\frac{2\pi}{p} \sum_{tuv} E_{tuv}^{ab} \sum_K Z_K R^0_{tuv}(p,\mathbf{R}_{PC_K})
$$

Before implementing the recursive formula for the auxiliary Hermite integrals, we are gonna implement the Boys function.

#### Boys function

The Boys function is related to many other special functions of mathematics, such as the _Euler gamma function_, the _lower incomplete gamma function_ or also the _error function_.
It turns out that the Boys function can be expressed as a special case of the _Kummer confluent hypergeometric function_ $M(a,b,x)$, in particular by the relation

$$
F_n(x) = \frac{M(n+\tfrac{1}{2},n+\tfrac{3}{2},-x)}{2n+1}
$$

It is particularly important that this function is efficiently coded, since it is used several times for the evaluation of the two-electron integrals (formally scaling as the fourth power of the number of primitives).
To this end, an efficient and readily accessible implementation of this function in Julia is provided by the [GSL](https://www.gnu.org/software/gsl/) library, accessible through a wrapper.  
You can install the wrapper as any other Julia package, by issuing `add GSL` in the package manager (accessed by typing `]` in the REPL).

Let us simply implement the Boys function as follows

In [30]:
import GSL: sf_hyperg_1F1

function boys(n::Int, x::Real)
    return sf_hyperg_1F1(n+0.5, n+1.5, -x) / (2.0*n+1.0)
end

boys (generic function with 1 method)

At this point we can implement the auxiliary Hermite Gaussian integrals

In [11]:
"""Returns the integral of an Hermite Gaussian divided by the Coulomb operator."""
function Rtuv(t::Int, u::Int, v::Int, n::Int, p::Real, RPC::Vector{T}) where {T<:Real}
    if t == u == v == 0
        return (-2.0*p)^n * boys(n,p*sum(RPC.*RPC))
    elseif u == v == 0
        if t > 1
            return  (t-1)*Rtuv(t-2, u, v, n+1, p, RPC) +
                   RPC[1]*Rtuv(t-1, u, v, n+1, p, RPC)
        else
            return RPC[1]*Rtuv(t-1, u, v, n+1, p, RPC)
        end
    elseif v == 0
        if u > 1
            return  (u-1)*Rtuv(t, u-2, v, n+1, p, RPC) +
                   RPC[2]*Rtuv(t, u-1, v, n+1, p, RPC)
        else
            return RPC[2]*Rtuv(t, u-1, v, n+1, p ,RPC)
        end
    else
        if v > 1
            return  (v-1)*Rtuv(t, u, v-2, n+1, p, RPC) +
                   RPC[3]*Rtuv(t, u, v-1, n+1, p, RPC)
        else
            return RPC[3]*Rtuv(t, u, v-1, n+1, p, RPC)
        end
    end
end

Rtuv

Let us now see how to implement the term $V_{ab}$.  
From a computational perspective, it is more convenient to carry out the Hermite-to-Cartesian transoformation for the three Cartesian components one after the other instead of all of them at the same time, i.e.

$$
\begin{aligned}
V_{ab} &= \frac{2\pi}{p} \sum_{tuv} E_{tuv}^{ab} R^0_{tuv}(p,\mathbf{R}_{PC}) \\ 
&=  \frac{2\pi}{p} \sum_{t} E_{t}^{ij} \sum_{u} E_{u}^{kl} \sum_{v} E_{v}^{mn} R^0_{tuv}(p,\mathbf{R}_{PC})
\end{aligned}
$$

The following function implements it

In [12]:
"""Returns the nuclear attraction energy integral of the distribution Ωab from center C."""
function attraction(Ga::CGF, Gb::CGF, C::Vector{T}) where {T<:Real}
    
    # precomputing all required quantities
    α = exponent(Ga); β = exponent(Gb)
    (i,k,m) = shell(Ga); (j,l,n) = shell(Gb)
    A = center(Ga); B = center(Gb)
    p = α + β
    P = (α*A + β*B) / (α + β)
    RAB = A - B; RPA = P - A; RPB = P - B; RPC = P - C
    μ = (α * β)/(α + β)
    Kαβ = exp.(-μ*RAB.^2)
    
    vne = 0.0
    for t = 0:i+j
        Eij = Etij(t, i, j, Kαβ[1], RPA[1], RPB[1], α, β)
        for u = 0:k+l
            Ekl = Etij(u, k, l, Kαβ[2], RPA[2], RPB[2], α, β)
            for v = 0:m+n
                Emn = Etij(v, m, n, Kαβ[3], RPA[3], RPB[3], α, β)
                vne +=  Eij * Ekl * Emn * Rtuv(t, u, v, 0, p, RPC)
            end
        end
    end
    
    return 2.0*π*vne/p
end

attraction

### Two-electron repulsion integrals

Finally we arrived at the most computational intensive and complicated integrals, the electron repulsion integrals (ERIs).  
However, we have already done all the hard work, i.e. the definition of the auxiliary Hermite integrals. Let's see, we are interested in the calculation of the following type of integrals

$$
g_{abcd} = \langle G_a G_b|\frac{1}{r_{12}}|G_c G_d \rangle = \int\int
\frac{\Omega_{ab}(\mathbf{r}_1)\Omega_{cd}(\mathbf{r}_2)}{r_{12}}d\mathbf{r}_1d\mathbf{r}_2
$$

Similarly to the previous case, we expand the Cartesian overlap distributions using Hermite Gaussians, such that we obtain

$$
g_{abcd} = \sum_{tuv} E_{tuv}^{ab} \sum_{\tau\nu\phi} E_{\tau\nu\phi}^{cd} \int\int \frac{\Lambda_{tuv}(\mathbf{r}_1)\Lambda_{\tau\nu\phi}(\mathbf{r}_2)}{r_{12}}d\mathbf{r}_1d\mathbf{r}_2
$$

We can express the double integral on the right-hand side using the auxiliary Hermite integrals $R_{tuv}^n$ derived precedently, such that the ERI formula is given by

$$
g_{abcd} = \frac{2\pi^{5/2}}{pq\sqrt{p+q}} \sum_{tuv} E_{tuv}^{ab} \sum_{\tau\nu\phi} (-1)^{\tau + \nu + \phi} E_{\tau\nu\phi}^{cd} R_{t+\tau,u+\nu,v+\phi}^n(\xi,\mathbf{R}_{PQ})
$$

where $p$ and $q$ are the exponent of the Hermite Gaussians centered in $\mathbf{P}$ and $\mathbf{Q}$, respectively.
The center $\mathbf{P}$ is computed according to

$$
\mathbf{P} = \frac{\alpha\mathbf{A} + \beta\mathbf{B}}{\alpha + \beta}
$$

and similarly is done for $\mathbf{Q}$, using the information of the GTOs $G_c$ and $G_d$.  
The exponent $\xi$ is called _reduced exponent_ and is given by

$$
\xi = \frac{pq}{p+q}
$$

So, at this point we can already implement the function computing the ERIs

In [13]:
"""Returns the two-electron integral gabcd = < Ga(r1) Gb(r1) | 1/r12 | Gc(r2) Gd(r2) >."""
function repulsion(Ga::CGF, Gb::CGF, Gc::CGF, Gd::CGF)
    
    # extract info from gaussians for electron 1
    α = exponent(Ga); β = exponent(Gb)
    (i1,k1,m1) = shell(Ga); (j1,l1,n1) = shell(Gb)
    A = center(Ga); B = center(Gb); RAB = A-B
    # precompute quantities for electron 1
    P = (α*A + β*B) / (α + β)
    p = α + β
    Kαβ = exp.(-α*β/(α+β)*RAB.^2)
    RPA = P-A; RPB = P-B
    
    # extract info from gaussians for electron 2
    γ = exponent(Gc); δ = exponent(Gd)
    (i2,k2,m2) = shell(Gc); (j2,l2,n2) = shell(Gd)
    C = center(Gc); D = center(Gd); RCD = C-D
    # precompute quantities for electron 2
    Q = (γ*C + δ*D) / (γ + δ)
    q = γ + δ
    Kγδ = exp.(-γ*δ/(γ+δ)*RCD.^2)
    RQC = Q-C; RQD = Q-D
    
    # precompute quantities for auxiliary integral R
    RPQ = P-Q
    ξ = p*q/(p+q)
    
    vee = 0.0
    for t = 0:i1+j1
        Eij1 = Etij(t, i1, j1, Kαβ[1], RPA[1], RPB[1], α, β)
        for u = 0:k1+l1
            Ekl1 = Etij(u, k1, l1, Kαβ[2], RPA[2], RPB[2], α, β)
            for v = 0:m1+n1
                Emn1 = Etij(v, m1, n1, Kαβ[3], RPA[3], RPB[3], α, β)
                for τ = 0:i2+j2
                    Eij2 = Etij(τ, i2, j2, Kγδ[1], RQC[1], RQD[1], γ, δ)
                    for ν = 0:k2+l2
                        Ekl2 = Etij(ν, k2, l2, Kγδ[2], RQC[2], RQD[2], γ, δ)
                        for ϕ = 0:m2+n2
                            Emn2 = Etij(ϕ, m2, n2, Kγδ[3], RQC[3], RQD[3], γ, δ)
                            vee += Eij1 * Ekl1 * Emn1 * Eij2 * Ekl2 * Emn2 *
                                   Rtuv(t+τ, u+ν, v+ϕ, 0, ξ, RPQ) * (-1)^(τ+ν+ϕ)
                        end
                    end
                end
            end
        end
    end
    
    return (vee*2.0*π^2.5)/(p*q*sqrt(p+q))
end

repulsion

And with this function we are done implementing the backbone of our molecular integrals program. We have all the functions to compute the various types of integrals of primitive Cartesian GTOs according to the McMurchie-Davidson scheme!

This again a good point to test our implementation so far

In [14]:
println("<Ga|T|Ga> = ",kinetic(Ga,Ga))  # -> 5.391510399487428
println("<Ga|T|Gb> = ",kinetic(Ga,Gb))  # -> 1.2081015154705197
println("<Gc|V|Gc> = ",attraction(Gc,Gc,center(Ga)))  # -> 11.986181257106331
println("<Gc|V|Gd> = ",attraction(Gc,Gd,center(Ga)))  # -> 0.28734166803518
println("(GcGc|GdGd) = ",repulsion(Gc,Gc,Gd,Gd))  # -> 4.249880629786412
println("(GaGb|GcGd) = ",repulsion(Ga,Gb,Gc,Gd))  # -> 0.14737599727691464

<Ga|T|Ga> = 5.391510399487428
<Ga|T|Gb> = 1.2081015154705197
<Gc|V|Gc> = 11.986181257106331
<Gc|V|Gd> = 0.28734166803518
(GcGc|GdGd) = 4.249880629786412
(GaGb|GcGd) = 0.14737599727691464


Now that the we are sure about the correctness of our code, we can continue.
As it was stated further above, Cartesian Gaussians are used to compute the integrals, but in general we would like to transform these integrals into a basis of spherical-harmonic Gaussians functions, i.e. SGFs, so let's defined them and see how to carry out that transformation!

## Spherical-harmonic Gaussian functions

Real-valued _spherical-harmonic Gaussian functions (SGFs)_, usually called _spherical-harmonic GTOs_, are real valued functions with the form

$$
G_{lm}(\mathbf{r},\alpha,\mathbf{A}) = S_{lm}(x_A,y_A,z_A) e^{-\alpha r_A^2}
$$

where $S_{lm}(\mathbf{r}_A)$ is one of the _real-valued solid harmonics_, which are formed by taking the real and imaginary parts of the _solid harmonics_ $\mathcal{Y}_{lm}(\mathbf{r}) = r^l Y_{lm}(\theta,\phi)$, with $Y_{lm}(\theta,\phi)$ being the usual _spherical harmonic functions_, i.e. the eigenfunctions of the total angular momentum operator squared $\hat{L}^2$ and the $z$-projection of the angular momentum $\hat{L}_z$.  
The position of the electron is defined relative to the nucleus at position $\mathbf{A}$

$$
\mathbf{r}_A = \mathbf{r} - \mathbf{A}
$$

and $x_A, y_A$ and $z_A$ are the relative Cartesian components. The _Gaussian exponent_ $\alpha$ is a real number greater than zero.
The total angular momentum is given by $l$, and the set of SGFs with $m=-l,-l+1,\ldots,l$ is said to constitute a _shell_.
The number of SGFs constituting a complete shell is given by

$$
N_l^s = 2l + 1
$$

Let us define a Julia class to represent such functions.

In [15]:
struct SGF <: BasisFunction
    A::NTuple{3,Float64}
    α::Float64
    l::Int
    m::Int
end

"""`SGF` constructor using a vector as the center."""
function SGF(A::AbstractVector{T}, α::Real, l::Int, m::Int) where {T<:Real}
    return SGF((A[1],A[2],A[3]),α,l,m)
end

center(gto::SGF)   = [gto.A[1],gto.A[2],gto.A[3]]
exponent(gto::SGF) = gto.α
shell(gto::SGF)    = gto.l
ml(gto::SGF)       = gto.m

ml (generic function with 1 method)

### Cartesian to spherical-harmonic transformation

A spherical-harmonic Gaussian $G_{lm}$ of angular momentum $l$ and projection $m$ can be expressed as a linear combination of Cartesian Gaussians $G_{ijk}$ with the same total angular momentum $l=i+j+k$ according to

$$
G_{lm}(\mathbf{r},\alpha,\mathbf{A}) = \sum_{i,j,k\\i+j+k=l} T_{ijk}^{lm} G_{ijk}(\mathbf{r},\alpha,\mathbf{A})
$$

This transformation is relatively easy in theory, but the formula for the transformation coefficients $T_{ijk}^{lm}$ is quite nasty, e.g.

$$
G_{lm}(\mathbf{r},\alpha,\mathbf{A}) =
N_{lm}^S \sum_{t=0}^{[(l-|m|)/2]}\sum_{u=0}^t\sum_{v=v_m}^{[|m|/2-v_m]+v_m}
C_{tuv}^{lm} G_{2t+|m|-2(u+v),2(u+v),l-2t-|m|}(\mathbf{r},\alpha,\mathbf{A})
$$

The numbers in the square brackets correspond to the largest integer less than or equal to the one inside (i.e. it rounds it down to the closest integer). The various terms appearing in the above expression read

\begin{align}
N_{lm}^S &= \frac{1}{2^{|m|}l!} \sqrt{\frac{2(l+|m|)!(l-|m|)!}{2^{\delta_{0m}}}} \\
C_{tuv}^{lm} &= (-1)^{t+v-v_m} \bigg( \frac{1}{4} \bigg)^t \binom{l}{t}
\binom{l-t}{|m|+t} \binom{t}{u} \binom{|m|}{2v} \\
v_m &= 
\begin{cases}
0 & m \ge 0 \\
\tfrac{1}{2} & m \lt 0
\end{cases}
\end{align}

Note that $v_m$ can be either $0$ or $\tfrac{1}{2}$ and that the third sum starts from $v=v_m$. This means that when $v_m$ is zero, the sum runs over integer numbers $v = 0, 1, 2, \ldots$, while when $v_m$ is equal one half, the sum runs over half integers $v = \tfrac{1}{2}, \tfrac{3}{2}, \ldots$.  
As we can see, the analytical form of SGFs is somewhat more complicated than that of CGFs, however, the coefficients $N_{lm}^S$, $C_{tuv}^{lm}$ and the index $v_m$ only depend on $l$ and $m$, thus they can be precomputed for all functions in a given basis set.

Let's move on to the implementation!  
First, we code the kernel functions that compute the coefficients $N_{lm}^S$ and $C_{tuv}^{lm}$

In [16]:
function NlmS(l::Int, m::Int)
    # factor in front of the sqrt
    fac = 1.0/(2^abs(m)*factorial(l))
    
    # numerator divided by half
    numhalf = factorial(l+abs(m))*factorial(l-abs(m))
    
    # check m for the denominator, multiply and return
    if m == 0
        return fac*sqrt(numhalf)
    else
        return fac*sqrt(2.0*numhalf)
    end
end


# note that `v` can take half integer values, thus is `Real`
function Ctuvlm(t::Int, u::Int, v::Real, l::Int, m::Int)
    # if statement to set v_m directly in the expression
    if m >= 0
        return (-1)^(t+v) * (0.25)^t * binomial(l,t) *
               binomial(l-t,abs(m)+t) * binomial(t,u) *
               binomial(abs(m),Int(2*v))
    else
        # note the -0.5 due to v_m
        return (-1)^(t+v-0.5) * (0.25)^t * binomial(l,t) *
               binomial(l-t,abs(m)+t) * binomial(t,u) *
               binomial(abs(m),Int(2*v))
    end
end

Ctuvlm (generic function with 1 method)

We now write a function that generates all the coefficients $T_{ijk}^{lm}$ for transformations up to an arbitrary angular momentum $l_{max}$

In [17]:
function Tijklm(lmax::Int)
    # initialize dictionary containing the transformation coeffs
    Tijklm_ = Dict{Tuple{Int,Int},Vector{Pair{Tuple{Int,Int,Int},Float64}}}()
    
    # loop over angular momenta
    for l = 0:lmax
        # loop over magnetic quantum numbers
        for m = -l:1:l
            # compute v_m
            if m < 0
                vm = 0.5
            else
                vm = 0.0
            end
            
            # since the loop over v runs with half integers
            # for m < 0, we run it instead over 2*v, where
            # vvmin and vvmax are beginning and end of the
            # loop over 2*v, such that this works for all
            # cases, i.e. also when v is an integer
            vvmin = Int(2 * vm)
            vvmax = Int(2 * (floor(Int,abs(m)/2-vm) + vm))
            
            # initialize vector holding coeffs
            Tlm = Vector{Pair{Tuple{Int,Int,Int},Float64}}()
            # loop over auxiliary indices
            for t = 0:floor(Int,(l-abs(m))/2)
                for u = 0:t
                    for vv = vvmin:2:vvmax
                        # compute v = vv/2, this might be
                        # either `Int` or `Float`
                        v = vv*0.5
                        # compute the Cartesian angular momenta
                        # to know the associated GTO to the
                        # transformation coefficient calculated here
                        i = Int(2*t+abs(m)-2*(u+v))
                        j = Int(2*(u+v))
                        k = Int(l-2*t-abs(m))
                        
                        # compute coeff and save it
                        T = NlmS(l,m)*Ctuvlm(t,u,v,l,m)
                        push!(Tlm,((i,j,k)=>T))
                    end
                end
            end
            push!(Tijklm_,(l,m) => Tlm)
        end
    end
    
    return Tijklm_
end

Tijklm (generic function with 1 method)

This function allows us to compute all coefficients up to an arbitrary angular momentum $l_{max}$ by calling `Tijklm(lmax)`, and access them using the Cartesian angular momenta `(i,j,k)` and the spherical ones `(l,m)`

In [18]:
# as an example, consider the case lmax=2
# first generate the coefficients
T = Tijklm(2)
println(T)

# access for instance the coeffs for l=2 and m=0
println("\naccessing arbitrary coeff:\n",T[(2,0)])

# length of T[(2,0)] tells me how many CGFs are needed
println("\n# of CGFs involved = ",length(T[(2,0)]))

println("iterate over the array and access coeffs")
for coeff in T[(2,0)]
    print("(i,j,k) = ",coeff.first)
    println("\tTijklm = ",coeff.second)
end

Dict((0, 0)=>[(0, 0, 0)=>1.0],(1, 0)=>[(0, 0, 1)=>1.0],(1, -1)=>[(0, 1, 0)=>1.0],(2, 0)=>[(0, 0, 2)=>1.0, (2, 0, 0)=>-0.5, (0, 2, 0)=>-0.5],(2, 2)=>[(2, 0, 0)=>0.866025, (0, 2, 0)=>-0.866025],(1, 1)=>[(1, 0, 0)=>1.0],(2, -2)=>[(1, 1, 0)=>1.73205],(2, -1)=>[(0, 1, 1)=>1.73205],(2, 1)=>[(1, 0, 1)=>1.73205])

accessing arbitrary coeff:
Pair{Tuple{Int64,Int64,Int64},Float64}[(0, 0, 2)=>1.0, (2, 0, 0)=>-0.5, (0, 2, 0)=>-0.5]

# of CGFs involved = 3
iterate over the array and access coeffs
(i,j,k) = (0, 0, 2)	Tijklm = 1.0
(i,j,k) = (2, 0, 0)	Tijklm = -0.5
(i,j,k) = (0, 2, 0)	Tijklm = -0.5


In practice, we will not transform the CGFs to SGFs directly, but rather we will compute integrals over CGFs and then transform them in the basis of SGFs.  
Let's see how such a transformation looks like

\begin{align}
\langle G_{lm} | \hat{O} | G_{l'm'} \rangle &= \langle
\sum_{\substack{i ,j, k \\i +j +k =l }} T_{ijk}^{lm}G_{ijk} | \hat{O} |
\sum_{\substack{i',j',k'\\i'+j'+k'=l'}} T_{i'j'k'}^{l'm'}G_{i'j'k'} \rangle \\ 
&= \sum_{\substack{i,j,k\\i+j+k=l}} \sum_{\substack{i',j',k'\\i'+j'+k'=l'}}
T_{ijk}^{lm} T_{i'j'k'}^{l'm'} \langle G_{ijk} | \hat{O} | G_{i'j'k'} \rangle
\end{align}

Note that since for an SGF with a given $l$ we need several CGFs with the same total $l=i+j+k$, it is best to calculate integrals shell by shell and not simply one by one, otherwise we might recompute the same integral over CGFs several times.
Nevertheless, for the purpose of this tutorial, we will not care about efficiency and we will actually generate every time on the fly all required CGFs for the integral that we want to compute.

In the following we'll start with the overlap, but the same can be done for all other types of integrals, and it will be mostly a copy & paste work.

In [19]:
"""Returns the overlap integral <Ga|Gb> of two spherical-harmonic GTOs."""
function overlap(Ga::SGF, Gb::SGF)
    # this returns an array telling us which functions we need
    Ta=T[(Ga.l,Ga.m)]
    Tb=T[(Gb.l,Gb.m)]
    
    # initialize arrays of CGFs
    CGFa=Vector{CGF}(undef,length(Ta))
    CGFb=Vector{CGF}(undef,length(Tb))
        
    # generate the functions
    for (it,ijk) in enumerate(Ta)
        CGFa[it]=CGF(Ga.A,Ga.α,ijk.first)
    end
    for (it,ijk) in enumerate(Tb)
        CGFb[it]=CGF(Gb.A,Gb.α,ijk.first)
    end
        
    # compute integrals in CGF basis and accumulate
    Sab = 0.0
    for (ia,CGa) in enumerate(CGFa)
        for (ib,CGb) in enumerate(CGFb)
            Sab += Ta[ia].second * Tb[ib].second * overlap(CGa,CGb)
        end
    end
    
    return Sab
end


"""Returns the self-overlap integral <Ga|Ga> of a `SGF` Ga."""
function overlap(Ga::SGF)
    return overlap(Ga,Ga)
end


"""Returns the normalization factor of a `SGF` Ga."""
function normalization(Ga::SGF)
    return 1.0/sqrt(overlap(Ga))
end

normalization

Let's do a quick test of our implementation.

In [20]:
# create two d shells with different exponent (STO-3G for Titanium)
GaShell = [SGF(zeros(3),0.502076728,2,m) for m=-2:2]
Na = normalization(GaShell[1])
GbShell = [SGF(zeros(3),0.193716810,2,m) for m=-2:2]
Nb = normalization(GbShell[1])

0.09310054997897052

In [25]:
# overlap between the two shells, only off-diag elements should be = 0
# diagonal entries should give ≈ 0.6820466292246176
for i=1:5
    for j=1:5
        print(Na*Nb*overlap(GaShell[i],GbShell[j]),"\t")
    end
    println()
end

0.6820466292246175	0.0	0.0	0.0	0.0	
0.0	0.6820466292246175	0.0	0.0	0.0	
0.0	0.0	0.6820466292246176	0.0	0.0	
0.0	0.0	0.0	0.6820466292246175	0.0	
0.0	0.0	8.151538340316786e-17	0.0	0.6820466292246175	


If you didn't do any mistake, you should get the same numbers as marked in the comments.
Let's now do the same for all other integrals. As I said above, this is mostly a copy & paste of the overlap function, simply changing the call to the other types of integrals.

In [22]:
"""Returns the kinetic energy integral <Ga|T|Gb> of two spherical-harmonic GTOs."""
function kinetic(Ga::SGF, Gb::SGF)
    # this returns an array telling us which functions we need
    Ta=T[(Ga.l,Ga.m)]
    Tb=T[(Gb.l,Gb.m)]
    
    # initialize arrays of CGFs
    CGFa=Vector{CGF}(undef,length(Ta))
    CGFb=Vector{CGF}(undef,length(Tb))
        
    # generate the functions
    for (it,ijk) in enumerate(Ta)
        CGFa[it]=CGF(Ga.A,Ga.α,ijk.first)
    end
    for (it,ijk) in enumerate(Tb)
        CGFb[it]=CGF(Gb.A,Gb.α,ijk.first)
    end
        
    # compute integrals in CGF basis and accumulate
    Tab = 0.0
    for (ia,CGa) in enumerate(CGFa)
        for (ib,CGb) in enumerate(CGFb)
            Tab += Ta[ia].second * Tb[ib].second * kinetic(CGa,CGb)
        end
    end
    
    return Tab
end


"""Returns the nuclear attraction energy integral of the distribution Ωab from center C."""
function attraction(Ga::SGF, Gb::SGF, C::Vector{Ty}) where {Ty<:Real}
    # this returns an array telling us which functions we need
    Ta=T[(Ga.l,Ga.m)]
    Tb=T[(Gb.l,Gb.m)]
    
    # initialize arrays of CGFs
    CGFa=Vector{CGF}(undef,length(Ta))
    CGFb=Vector{CGF}(undef,length(Tb))
        
    # generate the functions
    for (it,ijk) in enumerate(Ta)
        CGFa[it]=CGF(Ga.A,Ga.α,ijk.first)
    end
    for (it,ijk) in enumerate(Tb)
        CGFb[it]=CGF(Gb.A,Gb.α,ijk.first)
    end
        
    # compute integrals in CGF basis and accumulate
    Vab = 0.0
    for (ia,CGa) in enumerate(CGFa)
        for (ib,CGb) in enumerate(CGFb)
            Vab += Ta[ia].second * Tb[ib].second * attraction(CGa,CGb,C)
        end
    end
    
    return Vab
end


"""Returns the two-electron integral gabcd = < Ga(r1) Gb(r1) | 1/r12 | Gc(r2) Gd(r2) >."""
function repulsion(Ga::SGF, Gb::SGF, Gc::SGF, Gd::SGF)
    # this returns an array telling us which functions we need
    Ta=T[(Ga.l,Ga.m)]
    Tb=T[(Gb.l,Gb.m)]
    Tc=T[(Gc.l,Gc.m)]
    Td=T[(Gd.l,Gd.m)]
    
    # initialize arrays of CGFs
    CGFa=Vector{CGF}(undef,length(Ta))
    CGFb=Vector{CGF}(undef,length(Tb))
    CGFc=Vector{CGF}(undef,length(Tc))
    CGFd=Vector{CGF}(undef,length(Td))
        
    # generate the functions
    for (it,ijk) in enumerate(Ta)
        CGFa[it]=CGF(Ga.A,Ga.α,ijk.first)
    end
    for (it,ijk) in enumerate(Tb)
        CGFb[it]=CGF(Gb.A,Gb.α,ijk.first)
    end
    for (it,ijk) in enumerate(Tc)
        CGFc[it]=CGF(Gc.A,Gc.α,ijk.first)
    end
    for (it,ijk) in enumerate(Td)
        CGFd[it]=CGF(Gd.A,Gd.α,ijk.first)
    end
        
    # compute integrals in CGF basis and accumulate
    gabcd = 0.0
    for (ia,CGa) in enumerate(CGFa)
        for (ib,CGb) in enumerate(CGFb)
            for (ic,CGc) in enumerate(CGFc)
                for (id,CGd) in enumerate(CGFd)
                    gabcd += Ta[ia].second * Tb[ib].second * 
                             Tc[ic].second * Td[id].second *
                             repulsion(CGa,CGb,Cgc,Cgd)
                end
            end
        end
    end
    
    return gabcd
end

repulsion

In [28]:
# test that the functions work
for i=1:5
    for j=1:5
        print(Na*Nb*attraction(GaShell[i],GbShell[j],ones(3)),"\t")
    end
    println()
end

for i=1:5
    for j=1:5
        print(Na*Nb*kinetic(GaShell[i],GbShell[j]),"\t")
    end
    println()
end

0.3289066824341946	0.04415303241711899	-0.02040561086522047	0.04415303241711899	0.0	
0.04415303241711899	0.3289066824341946	0.010202805432610233	0.04415303241711899	-0.017671777389020676	
-0.02040561086522047	0.010202805432610233	0.30242542740609624	0.010202805432610235	0.0	
0.04415303241711899	0.04415303241711899	0.010202805432610235	0.3289066824341946	0.017671777389020676	
0.0	-0.017671777389020676	0.0	0.017671777389020676	0.3024254274060963	
0.6673737436678823	0.0	0.0	0.0	0.0	
0.0	0.6673737436678823	0.0	0.0	0.0	
0.0	0.0	0.6673737436678825	0.0	2.0378845850791965e-17	
0.0	0.0	0.0	0.6673737436678823	0.0	
0.0	0.0	-2.0378845850791965e-17	0.0	0.6673737436678824	


Now we have a simple (although not efficient) way to compute integrals over SGFs! We shall now move to the last part of this tutorial, where we will compute integrals over contracted GTOs!

## Contracted Gaussian functions

In electronic structure theory, the basis functions used for molecular calculations are usually composed by _fixed_ linear combinations over simple Gaussian-type orbitals centered on the atomic nuclei.
Such basis function are thus called _atomic orbitals (AOs)_ and are given by

$$
\chi_{\mu}(\mathbf{r}) = \sum_{a} d_{a\mu} G_a(\mathbf{r})
$$

Another common name for $\chi_{\mu}$ is _contraction_ whereas the GTOs $G_a$ are called _primitives_.
The coefficients $d_{a\mu}$ of the superposition are named the _contraction coefficients_.
Note that all primitives forming a single AO are centered on the same nucleus and all have the same angular momentum.

The contraction coefficients as well as the exponents of the primitives are usually preoptimized and do not change during a quantum chemical calculation. A large number of _standard_ basis sets are available which provide sets of _optimal_ coefficients and exponents.

In order to represent _contracted GTOs (CGTOs)_, we shall introduce a class defining this type of functions.

In [38]:
"""Contracted Gaussian-type orbital."""
struct CGTO{GTO<:BasisFunction}
    funcs::Vector{GTO}          # primitive GTOs
    coefs::Vector{Float64}      # contraction coefs
    norms::Vector{Float64}      # normalization factors
end


"""
Construct a `CGTO` centered on `A` with angular momenutm `l`.
If `l` is a tuple with 3 entries (lx,ly,lz), then this `CGTO` will be made of `CGF`,
if it has only 2 entries (l,m), then this `CGTO` will be made of `SGF`.
"""
function CGTO(A::AbstractVector{T}, l::Tuple{Int,Int,Int},
              α::AbstractVector{T}, d::AbstractVector{T} ) where {T<:Real}
    
    # number of coefs and exponents must match
    @assert(length(d) == length(α))
    n = length(d)
    
    # initialize data vector
    funcs = Vector{CGF}(n)
    norms = Vector{Float64}(n)
    
    for i in 1:n
        funcs[i] = CGF(A,α[i],l)
        norms[i] = normalization(funcs[i])
    end
    
    return CGTO(funcs, d, norms)
end


"""
Construct a `CGTO` centered on `A` with angular momenutm `l`.
If `l` is a tuple with 3 entries (lx,ly,lz), then this `CGTO` will be made of `CGF`,
if it has only 2 entries (l,m), then this `CGTO` will be made of `SGF`.
"""
function CGTO(A::AbstractVector{T}, l::Tuple{Int,Int},
              α::AbstractVector{T}, d::AbstractVector{T} ) where {T<:Real}
    
    # number of coefs and exponents must match
    @assert(length(d) == length(α))
    n = length(d)
    
    # check quantum numbers are valid
    @assert(abs(m) <= l && l >= 0)
    
    # initialize data vector
    funcs = Vector{SGF}(n)
    norms = Vector{Float64}(n)
    
    for i in 1:n
        funcs[i] = SGF(A,α[i],l,m)
        norms[i] = normalization(funcs[i])
    end
    
    return CGTO(funcs, d, norms)
end

primitives(cgto::CGTO) = cgto.funcs
nprims(cgto::CGTO) = length(cgto.funcs)
center(cgto::CGTO) = center(cgto.funcs[1])
shell(cgto::CGTO) = ijk(cgto.funcs[1])
ltot(cgto::CGTO) = ltot(cgto.funcs[1])
exponents(cgto::CGTO) = map(x->alpha(x),primitives(cgto))
coefs(cgto::CGTO) = cgto.coefs
norms(cgto::CGTO) = cgto.norms

norms (generic function with 1 method)

Note that until now we have worked with unnormalized primitives GTOs, but when dealing with contracted GTOs, it is more convenient to normalize them.
This is taken care of by the explicit constructor function defined above.

### One-electron integrals over contracted GTOs

The idea is now to define functions that compute the integrals over contracted GTOs by exploiting those implemented above for primitive GTOs.
In general, we have that such integrals are given by

$$
\begin{aligned}
O_{\mu\nu} &= \int \chi_{\mu}(\mathbf{r}) \hat{O}(\mathbf{r}) \chi_{\nu}(\mathbf{r}) d\mathbf{r} \\
&= <\chi_{\mu}|\hat{O}(\mathbf{r})|\chi_{\nu}> \\
&= <\sum_{a} d_{a\mu}^* G_a|\hat{O}(\mathbf{r})|\sum_{b} d_{b\nu} G_b> \\
&= \sum_{ab} d_{a\mu}^* d_{b\nu}<G_a|\hat{O}(\mathbf{r})|G_b>
\end{aligned}
$$

In practical terms however, we have to take into consideration the normalization coefficients of the primitives that we have neglected until now.
Consequently, the final general expression is given by

$$
\begin{aligned}
O_{\mu\nu} &= \sum_{ab} d_{a\mu}^* d_{b\nu} <G_a|\hat{O}(\mathbf{r})|G_b> \\
&= \sum_{ab} d_{a\mu}^* d_{b\nu} <N_a \tilde{G}_a|\hat{O}(\mathbf{r})|N_b \tilde{G}_b> \\
&= \sum_{ab} N_a N_b d_{a\mu}^* d_{b\nu} <\tilde{G}_a|\hat{O}(\mathbf{r})|\tilde{G}_b> \\
&= \sum_{ab} N_a N_b d_{a\mu}^* d_{b\nu} O_{ab} 
\end{aligned}
$$

where the functions $\tilde{G}_a$ and $\tilde{G}_b$ are the unnormalized GTOs that we have worked with so far and therefore $O_{ab}$ is the type of integrals that we can compute with the functions implemented until now.

In the following we implement all the one-electron integrals over CGTOs according to the simple double sum given on the right-hand side of last equation.

In [39]:
"""Returns the overlap integral between two contracted GTOs."""
function overlap(μ::CGTO, ν::CGTO)
    Gμ = primitives(μ); dμ = coefs(μ); Nμ = norms(μ)
    Gν = primitives(ν); dν = coefs(ν); Nν = norms(ν)
    S = 0.0
    for a in 1:nprims(μ)
        for b in 1:nprims(ν)
            S += Nμ[a] * Nν[b] * dμ[a] * dν[b] * overlap(Gμ[a],Gν[b])
        end
    end
    return S
end


"""Returns the kinetic energy integral between two contracted GTOs."""
function kinetic(μ::CGTO, ν::CGTO)
    Gμ = primitives(μ); dμ = coefs(μ); Nμ = norms(μ)
    Gν = primitives(ν); dν = coefs(ν); Nν = norms(ν)
    T = 0.0
    for a in 1:nprims(μ)
        for b in 1:nprims(ν)
            T += Nμ[a] * Nν[b] * dμ[a] * dν[b] * kinetic(Gμ[a],Gν[b])
        end
    end
    return T
end


"""Returns the nuclear attraction integral between two contracted GTOs and the nucleus centered at `C`."""
function nuclear(μ::CGTO, ν::CGTO, C::Vector{Real})
    Gμ = primitives(μ); dμ = coefs(μ); Nμ = norms(μ)
    Gν = primitives(ν); dν = coefs(ν); Nν = norms(ν)
    V = 0.0
    for a in 1:nprims(μ)
        for b in 1:nprims(ν)
            V += Nμ[a] * Nν[b] * dμ[a] * dν[b] * kinetic(Gμ[a],Gν[b],C)
        end
    end
    return V
end

nuclear

### Two-electron integrals over contracted GTOs

Analogously to the one-electron case, for the two-electron integrals we need to loop over the primitives of each atomic orbitals, but they are now four instead of two.

The corresponding implementation of the function is given below.

In [40]:
"""Returns the two-electron repulsion integral over four contracted GTOs."""
function repulsion(μ::CGTO, ν::CGTO, λ::CGTO, σ::CGTO)
    Gμ = primitives(μ); dμ = coefs(μ); Nμ = norms(μ)
    Gν = primitives(ν); dν = coefs(ν); Nν = norms(ν)
    Gλ = primitives(λ); dλ = coefs(λ); Nλ = norms(λ)
    Gσ = primitives(σ); dσ = coefs(σ); Nσ = norms(σ)
    V = 0.0
    for a in 1:nprims(μ)
        for b in 1:nprims(ν)
            for c in 1:nprims(λ)
                for d in 1:nprims(σ)
                    V += Nμ[a] * Nν[b] * Nλ[c] * Nσ[d] *
                         dμ[a] * dν[b] * dλ[c] * dσ[d] *
                         repulsion(Gμ[a],Gν[b],Gλ[c],Gσ[d])
                end
            end
        end
    end
    return V
end

repulsion

And with this function we are done computing integrals over contractions! All the hard work was actually already done before.

## Appendix

### Hermite Gaussian expansion coefficients from alternative recurrence relations

The Hermite Gaussian expansion coefficients can be obtained with a number of recursive relations. The ones presented above turn out to be the most efficient and easiest to implement. However, before deciding for to employ those relations, I tried out a different set.
In particular, the recursion given by

$$
\begin{aligned}
    E_t^{ij} &= 0 \, , \quad t < 0 \quad \text{or} \quad t > i+j \\
    E_t^{i+1,j} &= \frac{1}{2p}E_{t-1}^{ij} + X_{PA}E_t^{ij} + (t+1)E_{t+1}^{ij} \\
    E_t^{i,j+1} &= \frac{1}{2p}E_{t-1}^{ij} + X_{PB}E_t^{ij} + (t+1)E_{t+1}^{ij} \\
    E_0^{00} &= K_{\alpha\beta}^x
\end{aligned}
$$

which is a three-term recursion relation.
A simple Julia implementation is given by the following function.

In [ ]:
"""
Returns the Hermite expansion coefficients for a 1D Cartesian overlap distribution
using a three-term recursion relation.
"""

function Etij_3tr(t::Int ,i::Int, j::Int, Kαβx, XPA, XPB, α, β)
    p = α + β
    if t < 0 || t > i+j
        return 0.0
    elseif i == j == t == 0
        return Kαβx
    elseif j == 0
        return (1/(2*p)) * Etij_3tr(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                     XPA * Etij_3tr(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                   (t+1) * Etij_3tr(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
    else
        return (1/(2*p)) * Etij_3tr(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                     XPB * Etij_3tr(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                   (t+1) * Etij_3tr(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
    end
end

### Everything from here down is outdated...

As in this formula above we compute a lot of zeros (ending up in the first `if` branch), I figured one could explicitly expand the relations for the coefficients (at least for the lowest angular momenta) and thus avoid the computation of zeros.
After tedious algebraic manipulation, one can get the following explicit form for the expansion coefficients necessary to compute overalps up to $l = 2$, i.e. the $d$ shell.

$$
\begin{aligned}
    \hline
    E_0^{00} &= K_{\alpha\beta}^x \\
    \hline
    E_0^{01} &= X_{PB} E_0^{00} \\
    E_1^{01} &= E_1^{01} = \frac{1}{2p} E_0^{00} \\
    \hline
    E_0^{10} &= X_{PA} E_0^{00} \\
    E_1^{10} &= E_1^{01} = \frac{1}{2p} E_0^{00} \\
    \hline
    E_0^{11} &= \left( X_{PA}X_{PB} + \frac{1}{2p} \right) E_0^{00} \\
    E_1^{11} &= \frac{X_{PA} + X_{PB}}{2p} E_0^{00} \\
    E_2^{11} &= E_2^{02} = E_2^{20} = \frac{1}{4p^2} E_0^{00} \\
    \hline
    E_0^{02} &= \left( X_{PB}^2 + \frac{1}{2p} \right) E_0^{00} \\
    E_1^{02} &= \frac{X_{PB}}{p} E_0^{00} \\
    E_2^{02} &= E_2^{11} = E_2^{20} = \frac{1}{4p^2} E_0^{00} \\
    \hline
    E_0^{20} &= \left( X_{PA}^2 + \frac{1}{2p} \right) E_0^{00} \\
    E_1^{20} &= \frac{X_{PA}}{p} E_0^{00} \\
    E_2^{20} &= E_2^{11} = E_2^{02} = \frac{1}{4p^2} E_0^{00} \\
    \hline
    E_0^{12} &= \left( X_{PA}X_{PB}^2 + \frac{X_{PA}}{2p} + \frac{X_{PB}}{p} \right) E_0^{00} \\
    E_1^{12} &= \left( \frac{X_{PB}^2}{2p} + \frac{X_{PA}X_{PB}}{p} + \frac{3}{4p^2} \right) E_0^{00} \\
    E_2^{12} &= \left( \frac{X_{PB}}{2p^2} + \frac{X_{PA}}{4p^2} \right) E_0^{00} \\
    E_3^{12} &= E_3^{21} = \frac{1}{8p^3} E_0^{00} \\
    \hline
    E_0^{21} &= \left( X_{PB}X_{PA}^2 + \frac{X_{PB}}{2p} + \frac{X_{PA}}{p} \right) E_0^{00} \\
    E_1^{21} &= \left( \frac{X_{PA}^2}{2p} + \frac{X_{PB}X_{PA}}{p} + \frac{3}{4p^2} \right) E_0^{00} \\
    E_2^{21} &= \left( \frac{X_{PA}}{2p^2} + \frac{X_{PB}}{4p^2} \right) E_0^{00} \\
    E_3^{21} &= E_3^{12} = \frac{1}{8p^3} E_0^{00} \\
    \hline
    E_0^{22} &= \left( X_{PB}^2X_{PA}^2 + \frac{2X_{PB}X_{PA}}{p} + \frac{X_{PB}^2}{2p} + \frac{X_{PA}^2}{2p} + \frac{3}{4p^2} \right) E_0^{00} \\
    E_1^{22} &= \left( \frac{X_{PB}X_{PA}^2}{p} + \frac{X_{PB}^2X_{PA}}{p} + \frac{3X_{PA}}{2p^2} + \frac{3X_{PB}}{2p^2} \right) E_0^{00} \\
    E_2^{22} &= \left( \frac{X_{PB}X_{PA}}{p^2} + \frac{X_{PA}^2}{4p^2} + \frac{X_{PB}^2}{4p^2} + \frac{3}{4p^3} \right) E_0^{00} \\
    E_3^{22} &= \left( \frac{X_{PA}}{4p^3} + \frac{X_{PB}}{4p^3} \right) E_0^{00} \\
    E_4^{22} &= \frac{1}{16p^4} E_0^{00} \\
    \hline
\end{aligned}
$$

which I then implemented in the following function.

In [ ]:
"""
Returns the Hermite expansion coefficients for a 1D Cartesian overlap distribution
using a three-term recursion relation with hard-coded coefficients up to l = 2.
"""
function Etij_3te(t::Int ,i::Int, j::Int, Kαβx, XPA, XPB, α, β)
    if t > i + j
        return 0.0
    end
    p = α + β
    if t == 0
        if i == j == 0
            return Kαβx
        elseif i == 0 && j == 1
            return XPB*Kαβx
        elseif i == 1 && j == 0
            return XPA*Kαβx
        elseif i == 1 && j == 1
            return (XPA*XPB + 1/(2*p))*Kαβx
        elseif i == 0 && j == 2
            return (XPB^2 + 1/(2*p))*Kαβx
        elseif i == 2 && j == 0
            return (XPA^2 + 1/(2*p))*Kαβx
        elseif i == 1 && j == 2
            return (XPA*XPB^2 + XPA/(2*p) + XPB/p)*Kαβx
        elseif i == 2 && j == 1
            return (XPB*XPA^2 + XPB/(2*p) + XPA/p)*Kαβx
        elseif i == 2 && j == 2
            return (XPA^2*XPB^2 + 2*XPA*XPB/p + XPA^2/(2*p) + XPB^2/(2*p) + 3/(4*p^2))*Kαβx
        elseif i > 2
            return   XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                   (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return   XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                   (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    elseif t == 1
        if i == 0 && j == 1
            return Kαβx/(2*p)
        elseif i == 1 && j == 0
            return Kαβx/(2*p)
        elseif i == 1 && j == 1
            return ( (XPA + XPB)/(2*p) )*Kαβx
        elseif i == 0 && j == 2
            return XPB*Kαβx/p
        elseif i == 2 && j == 0
            return XPA*Kαβx/p
        elseif i == 1 && j == 2
            return (XPA*XPB/p + XPB^2/(2*p) + 3/(4*p^2))*Kαβx
        elseif i == 2 && j == 1
            return (XPA*XPB/p + XPA^2/(2*p) + 3/(4*p^2))*Kαβx
        elseif i == 2 && j == 2
            return (XPA*XPB^2/p + XPB*XPA^2/p + 3*XPA/(2*p^2) + 3*XPB/(2*p^2))*Kαβx
        elseif i > 2
            return (1/(2*p)) * Etij_3te(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                         XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return (1/(2*p)) * Etij_3te(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                         XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    elseif t == 2
        if i == 1 && j == 1
            return Kαβx/(4*p^2)
        elseif i == 0 && j == 2
            return Kαβx/(4*p^2)
        elseif i == 2 && j == 0
            return Kαβx/(4*p^2)
        elseif i == 1 && j == 2
            return (XPB/(2*p^2) + XPA/(4*p^2))*Kαβx
        elseif i == 2 && j == 1
            return (XPA/(2*p^2) + XPB/(4*p^2))*Kαβx
        elseif i == 2 && j == 2
            return (XPA*XPB/(p^2) + XPA^2/(4*p^2) + XPB^2/(4*p^2) + 3/(4*p^3))*Kαβx
        elseif i > 2
            return (1/(2*p)) * Etij_3te(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                         XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return (1/(2*p)) * Etij_3te(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                         XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    elseif t == 3
        if i == 1 && j == 2
            return Kαβx/(8*p^3)
        elseif i == 2 && j == 1
            return Kαβx/(8*p^3)
        elseif i == 2 && j == 2
            return (XPA/(4*p^3) + XPB/(4*p^3))*Kαβx
        elseif i > 2
            return (1/(2*p)) * Etij_3te(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                         XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return (1/(2*p)) * Etij_3te(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                         XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    elseif t == 4
        if i == 2 && j == 2
            return Kαβx/(16*p^4)
        elseif i > 2
            return (1/(2*p)) * Etij_3te(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                         XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return (1/(2*p)) * Etij_3te(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                         XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    elseif t > 4
        if i > 2
            return (1/(2*p)) * Etij_3te(t-1, i-1, j, Kαβx, XPA, XPB, α, β) +
                         XPA * Etij_3te(t  , i-1, j, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i-1, j, Kαβx, XPA, XPB, α, β)
        else
            return (1/(2*p)) * Etij_3te(t-1, i, j-1, Kαβx, XPA, XPB, α, β) +
                         XPB * Etij_3te(t  , i, j-1, Kαβx, XPA, XPB, α, β) +
                       (t+1) * Etij_3te(t+1, i, j-1, Kαβx, XPA, XPB, α, β)
        end
    end
end

I know, I realize this implementation is considerbly more lengthy and cumbersome than the recursive one, but it will hopefully perform better.

Let us have a look at what we have gained in terms of computational time.  
In the following we will use the very handy [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) package to measure performance. Just download it as any other package by issuing `Pkg.add("BenchmarkTools")` in the REPL.

In [ ]:
# load the package
using BenchmarkTools

# define some random input variables
α = 0.5
β = 0.75
μ = (α * β)/(α + β)
_Ax = 0.0
_Bx = 0.5
Px = (_Ax*α + _Bx*β)/(α + β)
XAB = _Ax-_Bx
XPA = Px-_Ax
XPB = Px-_Bx
Kαβx = exp(-μ*XAB^2)

# define some angular momenta
i=1;j=2

In [ ]:
# three-term recursive
@benchmark for t = 0:i+j
    Etij_3tr(t, $i, $j, $Kαβx, $XPA, $XPB, $α, $β)
end

In [ ]:
# three-term explicit
@benchmark for t = 0:i+j
    Etij_3te(t, $i, $j, $Kαβx, $XPA, $XPB, $α, $β)
end

In [ ]:
# two-term recursive
@benchmark for t = 0:i+j
    Etij(t, $i, $j, $Kαβx, $XPA, $XPB, $α, $β)
end

In [ ]:
using PyCall
@pyimport scipy.special as ss

# super slow
function pyboys(n, x)
    return ss.hyp1f1(n+0.5, n+1.5, -x) / (2.0*n+1.0)
end

# this should be correct, but seems a bad approximation
function incgamma(n, x)
    # only case we are going to use
    assert(x>0 && x < n + 1.5)
    
    term = x^n * exp(-x) / n
    incg = term
    den = n
    i = 1
    maxiter = 100
    tol = 1e-25
    
    while (abs(term) > tol && i < maxiter)
        den += i
        term *= (x / den)
#         println(term)
        incg += term
        i += 1
    end
#     println(i)
    return incg
end

# this is not working
function myboys(n, x)
    if x < n + 1.5
        b = n + 0.5
        if x < 1e-16
            return 0.5/b
        else
            return sf_gamma_inc(b,x)/(2*x^b)
#             return incgamma(b,x)/(2*x^b)
        end
    else
        i = n
        acc = 1.0
        if n > 0
            b = 0.5/x
            emx = exp(-x)
            while i > 0 
                acc = b * ((2*i-1) * acc - emx)
                i -= 1
            end
        end
        return acc*sqrt(pi/(4*x))*erf(sqrt(x))
     end
end

# this does not work either
function myboys2(n::Int, x)
    
    f = Vector{Float64}(n+1)
    
    if x < n + 1.5
        
        b = n + 0.5
        t = 1
        s = 1
        e = .5 * exp(-x)
        
        if x < 1e-16
            f[n+1] = 0.5/b
        else
            i = 1
            while t > 1e-16
                t *= x / (b + i)
                s += x
                i += 1
            end
            
            f[n+1] = e * s / b
            
            if (n > 0) 
                for i = n:-1:1
                    b -= 1
                    f[i] = (e + x * f[i+1]) / b
                end
            end
        end
    else
        f[1] = sqrt(pi/(4*x))*erf(sqrt(x))
        i = 1
        if n > 0
            b = 0.5/x
            e = exp(-x)
            while i <= n 
                f[i+1] = b* ( (2*i-1) * f[i] - e)
                i += 1
            end
        end
     end
    return f
end


using SpecialFunctions

# this only works for n >= 3//2
function gamma_inc(n::Rational{Int}, x)
    acc = sum([ x^(j+0.5)/( (j+0.5) * gamma(j+0.5)) for j = 0:n-3//2])
    return gamma(n)*(erfc(sqrt(x)) + e^(-x) * acc )
end